In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
import os

project_path = "/content/drive/MyDrive/ds_miloni_halkati"
os.chdir(project_path)

print("Current Directory:", os.getcwd())
print("Files:", os.listdir())
print("CSV Folder:", os.listdir("csv_files"))

Current Directory: /content/drive/MyDrive/ds_miloni_halkati
Files: ['csv_files', 'outputs', 'notebook_1.ipynb']
CSV Folder: ['historical_data.csv', 'fear_greed_index.csv']


In [4]:
import pandas as pd
import numpy as np

# Paths to your datasets inside Drive
fear_path = "csv_files/fear_greed_index.csv"
hist_path = "csv_files/historical_data.csv"

# Load both datasets
fear = pd.read_csv(fear_path)
hist = pd.read_csv(hist_path)

print("=== Fear & Greed Dataset ===")
display(fear.head())
print(fear.dtypes)

print("\n=== Historical Trader Dataset ===")
display(hist.head())
print(hist.dtypes)

=== Fear & Greed Dataset ===


,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


timestamp          int64
value              int64
classification    object
date              object
dtype: object

=== Historical Trader Dataset ===


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


Account              object
Coin                 object
Execution Price     float64
Size Tokens         float64
Size USD            float64
Side                 object
Timestamp IST        object
Start Position      float64
Direction            object
Closed PnL          float64
Transaction Hash     object
Order ID              int64
Crossed                bool
Fee                 float64
Trade ID            float64
Timestamp           float64
dtype: object


In [5]:
fear.columns = fear.columns.str.strip().str.lower().str.replace(" ", "_")
hist.columns = hist.columns.str.strip().str.lower().str.replace(" ", "_")

print("Fear columns:", fear.columns.tolist())
print("Hist columns:", hist.columns.tolist())

Fear columns: ['timestamp', 'value', 'classification', 'date']
Hist columns: ['account', 'coin', 'execution_price', 'size_tokens', 'size_usd', 'side', 'timestamp_ist', 'start_position', 'direction', 'closed_pnl', 'transaction_hash', 'order_id', 'crossed', 'fee', 'trade_id', 'timestamp']


In [6]:
# Convert UNIX timestamp → datetime
fear['timestamp'] = pd.to_numeric(fear['timestamp'], errors='coerce')
fear['datetime_from_timestamp'] = pd.to_datetime(fear['timestamp'], unit='s')

# Convert the 'date' column
fear['date'] = pd.to_datetime(fear['date'], errors='coerce')

# Create date-only column for merge
fear['date_only'] = fear['date'].dt.date

fear.head()

,timestamp,value,classification,date,datetime_from_timestamp,date_only
0,1517463000,30,Fear,2018-02-01,2018-02-01 05:30:00,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02,2018-02-02 05:30:00,2018-02-02
2,1517635800,40,Fear,2018-02-03,2018-02-03 05:30:00,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04,2018-02-04 05:30:00,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05,2018-02-05 05:30:00,2018-02-05


In [7]:
print(hist.columns)

Index(['account', 'coin', 'execution_price', 'size_tokens', 'size_usd', 'side',
       'timestamp_ist', 'start_position', 'direction', 'closed_pnl',
       'transaction_hash', 'order_id', 'crossed', 'fee', 'trade_id',
       'timestamp'],
      dtype='object')


In [8]:
# Parse timestamp_ist first (human-readable)
hist['timestamp_ist'] = pd.to_datetime(hist['timestamp_ist'], errors='coerce')

# Fallback: parse UNIX timestamp if IST missing
hist['timestamp'] = pd.to_numeric(hist['timestamp'], errors='coerce')
hist['timestamp_unix'] = pd.to_datetime(hist['timestamp'], unit='ms', errors='coerce')

# Create date_only for merging
hist['date_only'] = hist['timestamp_ist'].dt.date
hist.loc[hist['date_only'].isna(), 'date_only'] = hist['timestamp_unix'].dt.date

In [9]:
fear['sentiment_simple'] = fear['classification'].replace({
    'Extreme Fear': 'Fear',
    'Fear': 'Fear',
    'Neutral': 'Neutral',
    'Greed': 'Greed',
    'Extreme Greed': 'Greed'
})

In [10]:
print("Fear sentiment distribution:")
print(fear['sentiment_simple'].value_counts())

print("\nTrader dataset timestamp success:")
print(hist['date_only'].isna().mean())

Fear sentiment distribution:
sentiment_simple
Fear       1289
Greed       959
Neutral     396
Name: count, dtype: int64

Trader dataset timestamp success:
0.0


In [11]:
merged = hist.merge(
    fear[['date_only', 'value', 'classification', 'sentiment_simple']],
    how='left',
    on='date_only'
)

print("Merged shape:", merged.shape)
display(merged.head())

Merged shape: (211224, 21)


,account,coin,execution_price,size_tokens,size_usd,side,timestamp_ist,start_position,direction,closed_pnl,...,order_id,crossed,fee,trade_id,timestamp,timestamp_unix,date_only,value,classification,sentiment_simple
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,2024-02-12 22:50:00,0.000000,Buy,0.0,...,52017706630,True,0.345404,8.950000e+14,1.730000e+12,2024-10-27 03:33:20,2024-02-12,70.0,Greed,Greed
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,2024-02-12 22:50:00,986.524596,Buy,0.0,...,52017706630,True,0.005600,4.430000e+14,1.730000e+12,2024-10-27 03:33:20,2024-02-12,70.0,Greed,Greed
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,2024-02-12 22:50:00,1002.518996,Buy,0.0,...,52017706630,True,0.050431,6.600000e+14,1.730000e+12,2024-10-27 03:33:20,2024-02-12,70.0,Greed,Greed
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,2024-02-12 22:50:00,1146.558564,Buy,0.0,...,52017706630,True,0.050043,1.080000e+15,1.730000e+12,2024-10-27 03:33:20,2024-02-12,70.0,Greed,Greed
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,2024-02-12 22:50:00,1289.488521,Buy,0.0,...,52017706630,True,0.003055,1.050000e+15,1.730000e+12,2024-10-27 03:33:20,2024-02-12,70.0,Greed,Greed


In [12]:
merged.to_csv("csv_files/trader_with_sentiment.csv", index=False)
print("Saved: csv_files/trader_with_sentiment.csv")

Saved: csv_files/trader_with_sentiment.csv
